In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


In [2]:
# df1 = pd.read_csv('test.csv')

# df_ben = pd.read_csv('replicated_test.csv')

# df_ben['w_parameters_tone'] = df1['parameters_tone']
# df_ben['w_parameters_prompt_template_name'] = df1['parameters_prompt_template_name']

# df_ben.to_csv('replicated_test.csv', index=False)

In [3]:
# df_ben = pd.read_csv('train.csv')

df_test = pd.read_csv('replicated_train.csv')
df_test

,id,user_edited,font,image,caption,parameters_chapter_title,parameters_chapter_summary,parameters_tone,created_at,user_id,...,parameters_prompt_template_name,parameters_photo_search_term,user_created_at,approved,has_logo,w_parameters_tone,w_parameters_prompt_template_name,same_caption,same_parameters_chapter_title,same_parameters_chapter_summary
0,38c3f9e9-1b3c-4eaa-9ddf-57b477b20cf5,False,1,https://marky-image-posts.s3.amazonaws.com/0c6...,🌟 Break free from self-doubt! In fashion entre...,The Birth of Stout Baas,"Follow the journey of how Stout Baas, a self m...",1,11,1,...,1,1,11,False,True,promotional,reminder-limiting-belief,31,3,24
1,0782787f-b214-49bc-a2c6-bbb77fbaf1c5,False,0,https://marky-image-posts.s3.amazonaws.com/568...,Express your unique style with our 10k Hip Hop...,Jewelry as a Form of Self-Expression,Delve into the role of jewelry as a form of se...,1,15,1,...,1,1,15,False,True,promotional,why-do-this,24,4,22
2,a7de4d69-bf31-456f-b853-d8ef0df319aa,False,0,https://marky-image-posts.s3.amazonaws.com/f46...,Ready to break free from the 9-5 grind? Discov...,Finding Your Muse,Uncover your true passions and discover how to...,1,23,1,...,1,1,15,False,True,informational,goal-advice,38,3,22
3,9070b324-32b7-4da5-92a7-4e923eab9103,False,0,https://marky-image-posts.s3.amazonaws.com/0fa...,"Introducing Rays Away, the game-changer in sun...",Targeting Eco-Conscious Individuals,Learn about the specific target audience of Ra...,1,0,1,...,1,1,14,False,False,inspirational,did-you-know-2-cta,34,3,20
4,55a8b74b-09b6-4e7f-858c-d9bc44215a95,False,0,https://marky-image-posts.s3.amazonaws.com/e49...,Struggling with Twitter ghostwriting? 🤔 Don't ...,Overcoming Challenges and Staying Motivated,Navigate the challenges and obstacles that com...,1,9,1,...,1,1,9,False,False,informational,stop-counterproductive-2,29,5,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6571,4ef0359d-42e2-423b-b224-a827acb166dd,False,0,https://marky-image-posts.s3.amazonaws.com/c4f...,📢 Last chance for sign enthusiasts! Explore ou...,signing,i sell products of sign,1,11,91,...,1,1,11,False,False,promotional,its-not-too-late,23,0,5
6572,7cd1f278-a5fe-44a9-929d-b97bc4e19098,False,0,https://marky-image-posts.s3.us-east-2.amazona...,Join us at [dance school name] where age is ju...,Jazzing it Up,Immerse yourself in the energetic world of jaz...,1,7,66,...,1,0,6,True,False,informational,solution-content-prompt,40,3,29
6573,995fe6ad-3972-43f6-8547-e0f4cad6134a,False,0,https://marky-image-posts.s3.amazonaws.com/dc9...,🌟 Struggling with fitness routine? No worries!...,Creating a Sustainable Health and Fitness Routine,Develop a personalized plan for long-term heal...,1,21,60,...,1,1,20,False,False,informational,dont-worry-2,35,7,18
6574,7b1b6d75-da63-4580-96f2-1e4eb349af02,False,0,https://marky-image-posts.s3.amazonaws.com/791...,"Embrace diversity, expand horizons! Fostering ...",The Power of Cultural Understanding,Explore how cultural understanding can bridge ...,1,17,134,...,1,1,17,True,True,inspirational,better-did-you-know,21,5,17


In [4]:
df_main = pd.read_csv('replicated_train.csv')

X = df_main.drop(['approved', # Permanent
                  'image', # Permanent
                #   'font',# Ben
                  'id',
                  'caption',
                  'parameters_chapter_title',
                  'parameters_chapter_summary',
                  # 'created_at',
                  # 'user_id',
                #   'parameters_prompt_template_name',# Ben
                #   'parameters_photo_search_term',# Ben
                  # 'user_created_at',
                  ], axis=1)

y = df_main['approved']

# DROPPING [image, ]

has_NaN = {}
for column in df_main.columns:
    nan_count = 0
    for i in range(df_main[column].size):
        if(pd.isna(df_main[column][i])):
            # print("hi")
            nan_count += 1
        
    # if nan_count > 0:
    has_NaN[column] = nan_count
        
        # print(f"Column '{column}' has {nan_count} NaN values.")

print(df_main.shape)
has_NaN

(6576, 22)


{'id': 0,
 'user_edited': 0,
 'font': 0,
 'image': 0,
 'caption': 0,
 'parameters_chapter_title': 1,
 'parameters_chapter_summary': 46,
 'parameters_tone': 0,
 'created_at': 0,
 'user_id': 0,
 'parameters_switchboard_template_name': 0,
 'parameters_theme': 284,
 'parameters_prompt_template_name': 0,
 'parameters_photo_search_term': 0,
 'user_created_at': 0,
 'approved': 0,
 'has_logo': 0,
 'w_parameters_tone': 219,
 'w_parameters_prompt_template_name': 219,
 'same_caption': 0,
 'same_parameters_chapter_title': 0,
 'same_parameters_chapter_summary': 0}

In [5]:
# df_main
X.head()
X['parameters_tone'].to_csv('hi.csv')

In [11]:
# Convert NaN to string
for column in X.columns:
    for i in range(X[column].size):
        if(pd.isna(X[column][i])):
            X.loc[i, column] = "NaN"
            
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=5)
categorical_columns = ['user_edited', 'parameters_switchboard_template_name', 'parameters_theme', 'has_logo', 'w_parameters_tone', 'w_parameters_prompt_template_name',]

model = CatBoostClassifier(iterations=530, depth=6, learning_rate=0.1, cat_features=categorical_columns) # 200 is best
model.fit(X_train, y_train, verbose=100)

y_test = y_test.tolist()

y_pred = model.predict(X_test).tolist()
for i in range(len(y_pred)): # Convert to boolean
    if y_pred[i] == 'True':
        y_pred[i] = True
    elif y_pred[i] == 'False':
        y_pred[i] = False

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
importance = model.feature_importances_
print(f'Accuracy: {accuracy}')

importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': importance})
importance_df = importance_df.sort_values(by='Importance', ascending=False)
print(importance_df)

# print(report)


0:	learn: 0.6704506	total: 39.8ms	remaining: 21.1s
100:	learn: 0.4340185	total: 5.15s	remaining: 21.9s
200:	learn: 0.3687059	total: 11.2s	remaining: 18.4s
300:	learn: 0.3260769	total: 16.2s	remaining: 12.3s
400:	learn: 0.2914844	total: 21.3s	remaining: 6.85s
500:	learn: 0.2595294	total: 26.2s	remaining: 1.51s
529:	learn: 0.2516029	total: 27.6s	remaining: 0us
Accuracy: 0.7886467308667005
                                 Feature  Importance
9                        user_created_at   14.442593
6                       parameters_theme   11.578603
3                             created_at   11.492132
12     w_parameters_prompt_template_name   10.436997
13                          same_caption   10.380269
4                                user_id    9.772180
15       same_parameters_chapter_summary    8.369771
14         same_parameters_chapter_title    6.741754
10                              has_logo    5.126618
11                     w_parameters_tone    4.826070
0                          

In [7]:
# plt.figure(figsize=(50,3))
# plt.plot(y_test,'o')
# plt.plot(y_pred,'x')

In [8]:
df_sol = pd.read_csv('replicated_test.csv')

# Convert NaN to string
for column in df_sol.columns:
    for i in range(df_sol[column].size):
        if(pd.isna(df_sol[column][i])):
            df_sol.loc[i, column] = "NaN"

df_sol = df_sol.drop(['image', 'id', 'caption',
                  'parameters_chapter_title',
                  'parameters_chapter_summary',], axis=1)

answer_pred = model.predict(df_sol).tolist()

array = pd.array(answer_pred)
series = pd.Series(array)
series.to_csv('answer.csv',index=False)
answer_pred

['False',
 'False',
 'False',
 'False',
 'True',
 'False',
 'False',
 'False',
 'False',
 'True',
 'True',
 'True',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'True',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'True',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'True',
 'False',
 'True',
 'False',
 'False',
 'True',
 'False',
 'False',
 'False',
 'False',
 'True',
 'False',
 'False',
 'False',
 'True',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'True',
 'False',
 'False',
 'True',
 'False',
 'False',
 'False',
 'True',
 'True',
 'False',
 'False',
 'True',
 'False',
 'True',
 'False',
 'True',
 'False',
 'False',
 'False',
 'False',
 'True',
 'True',
 'False',
 'False',
 'False',
 'False',
 'True',
 'True',
 'True',
 'False',
 'False',
 'False',
 'False',
 'True',
 'False',
 'Tr